<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#닥터-브로너스-데이터-수집" data-toc-modified-id="닥터-브로너스-데이터-수집-1">닥터 브로너스 데이터 수집</a></span></li></ul></div>

In [27]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings('ignore')

https://drbronnerskorea.com/robots.txt 확인 결과 allow

https://drbronnerskorea.com/article/%EC%9D%B4%EC%9A%A9%EC%95%88%EB%82%B4-faq/3/2287/#none 에 따라 비건 인증을 받지 않은 오가닉 립밤, 오가닉 매직밤, 스킨 소프트너 상품 제외 후 데이터 수집

# 닥터 브로너스 데이터 수집

In [62]:
url = 'https://drbronnerskorea.com/product/list.html?cate_no=56'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [63]:
# 상품명 수집
item = html.find_all('span', style='font-size:13px;color:#555555;')
print(len(item))
item[:5]

48


[<span style="font-size:13px;color:#555555;">아몬드 퓨어 캐스틸 솝 950ml</span>,
 <span style="font-size:13px;color:#555555;">베이비 언센티드 퓨어 캐스틸 솝 950ml</span>,
 <span style="font-size:13px;color:#555555;">시트러스 오렌지 퓨어 캐스틸 솝 950ml</span>,
 <span style="font-size:13px;color:#555555;">유칼립투스 퓨어 캐스틸 솝 950ml</span>,
 <span style="font-size:13px;color:#555555;">라벤더 퓨어 캐스틸 솝 950ml</span>]

In [64]:
# 가격 수집 (정가)
price = html.find_all('span', class_='price')
print(len(price))
price[:5]

48


[<span class="price ">31,000원</span>,
 <span class="price ">31,000원</span>,
 <span class="price ">31,000원</span>,
 <span class="price ">31,000원</span>,
 <span class="price ">31,000원</span>]

In [65]:
# 가격 수집 (할인가)
d_price = html.find_all('span', class_='sale')
print(len(d_price))
d_price[:5]

48


[<span class="sale displaynone"></span>,
 <span class="sale displaynone"></span>,
 <span class="sale displaynone"></span>,
 <span class="sale displaynone"></span>,
 <span class="sale displaynone"></span>]

In [82]:
html.find('span', class_="xans-element- xans-product xans-product-headcategory ").text

'퓨어 캐스틸 솝 (매직솝)'

In [95]:
# 최종 스크래핑 코드
def cosmetic_scraping(category):
    
    total_data = []
    for cat in category:
        url = f'https://drbronnerskorea.com/product/list.html?cate_no={cat}'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        so_cat = html.find('span', class_="xans-element- xans-product xans-product-headcategory ").text
        item = html.find_all('span', style='font-size:13px;color:#555555;')
        price = html.find_all('span', class_='price')
        d_price = html.find_all('span', class_='sale')

        data = []
        for i in range(len(item)):
            item2 = item[i].text
            price2 = price[i].text
            d_price2 = d_price[i].text
            data.append([so_cat, item2, price2, d_price2])
        total_data.append(data)
    
    df = pd.DataFrame(total_data[1])
    df.columns = ['소분류명', '상품명', '정가', '할인가']
    return df

In [96]:
# 카테고리 선정 후 수집
df = cosmetic_scraping(['56','57','58','100','101','60'])
print(df.shape)
df.head()

(11, 4)


,소분류명,상품명,정가,할인가
0,퓨어 캐스틸 바솝,아몬드 퓨어 캐스틸 바솝 140g,"8,000원",
1,퓨어 캐스틸 바솝,베이비 마일드 퓨어 캐스틸 바솝 140g,"8,000원",
2,퓨어 캐스틸 바솝,시트러스 오렌지 퓨어 캐스틸 바솝 140g,"8,000원",
3,퓨어 캐스틸 바솝,유칼립투스 퓨어 캐스틸 바솝 140g,"8,000원",
4,퓨어 캐스틸 바솝,라벤더 퓨어 캐스틸 바솝 140g,"8,000원",


In [94]:
df

,소분류명,상품명,정가,할인가
0,퓨어 캐스틸 솝 (매직솝),아몬드 퓨어 캐스틸 솝 950ml,"31,000원",
1,퓨어 캐스틸 솝 (매직솝),베이비 언센티드 퓨어 캐스틸 솝 950ml,"31,000원",
2,퓨어 캐스틸 솝 (매직솝),시트러스 오렌지 퓨어 캐스틸 솝 950ml,"31,000원",
3,퓨어 캐스틸 솝 (매직솝),유칼립투스 퓨어 캐스틸 솝 950ml,"31,000원",
4,퓨어 캐스틸 솝 (매직솝),라벤더 퓨어 캐스틸 솝 950ml,"31,000원",
5,퓨어 캐스틸 솝 (매직솝),페퍼민트 퓨어 캐스틸 솝 950ml,"31,000원","24,800원"
6,퓨어 캐스틸 솝 (매직솝),로즈 퓨어 캐스틸 솝 950ml,"31,000원",
7,퓨어 캐스틸 솝 (매직솝),티트리 퓨어 캐스틸 솝 950ml,"31,000원",
8,퓨어 캐스틸 솝 (매직솝),체리블라썸 퓨어 캐스틸 솝 950ml,"31,000원",
9,퓨어 캐스틸 솝 (매직솝),샌달우드 자스민 퓨어 캐스틸 솝 950ml,"31,000원",


In [43]:
df.reset_index(drop=True, inplace=True)
df['브랜드명'] = '닥터브로너스'
df = df[['브랜드명','상품명','정가','할인가']]
df.head()

,브랜드명,상품명,정가,할인가
0,닥터브로너스,시트러스 오렌지 퓨어 캐스틸 솝 950ml,"31,000원",
1,닥터브로너스,티트리 퓨어 캐스틸 솝 950ml,"31,000원",
2,닥터브로너스,베이비 언센티드 퓨어 캐스틸 솝 475ml,"20,000원",
3,닥터브로너스,시트러스 오렌지 퓨어 캐스틸 솝 475ml,"20,000원",
4,닥터브로너스,라벤더 퓨어 캐스틸 솝 475ml,"20,000원",


In [55]:
drop_idx = df[df['상품명'].str.contains('립밤|매직밤|스킨|코코넛 밤')].index
df.drop(drop_idx, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(91, 4)


,브랜드명,상품명,정가,할인가
0,닥터브로너스,시트러스 오렌지 퓨어 캐스틸 솝 950ml,"31,000원",
1,닥터브로너스,티트리 퓨어 캐스틸 솝 950ml,"31,000원",
2,닥터브로너스,베이비 언센티드 퓨어 캐스틸 솝 475ml,"20,000원",
3,닥터브로너스,시트러스 오렌지 퓨어 캐스틸 솝 475ml,"20,000원",
4,닥터브로너스,라벤더 퓨어 캐스틸 솝 475ml,"20,000원",


In [57]:
df.to_csv('./cosmetic/닥터브로너스.csv', index=False)